In [8]:
from rlrom.trainers import RLTrainer
from rlrom.testers import RLTester
import os, sys, glob
from rlrom.utils import *
from pprint import pprint

In [2]:
cfg_file = 'cartpole/cfg0tr_ppo.yml'
#cfg_file = 'highway_env/cfg0tr.yml'
#T = RLTrainer(cfg_file)
cfg = load_cfg(cfg_file)


loading field [ ppo ] from YAML file [ cartpole_ppo.yml ]


In [3]:
print(cfg.get('this_cfg_pathdir'))
mpath,_ = get_model_fullpath(cfg)
print(mpath)
patt = os.path.splitext(mpath)[0]
print(glob.glob(patt))    


/home/alex/workspace_local/rlrom/examples/cartpole
/home/alex/workspace_local/rlrom/examples/cartpole/models/vanilla_ppo.zip
[]


In [10]:
def get_training_folders(cfg):
    cfg= load_cfg(cfg)
    mpath,_ = get_model_fullpath(cfg)
    patt = os.path.splitext(mpath)[0]
    globs = glob.glob(patt+'*')
    folders = []
    for d in globs:
        if os.path.isdir(d):
            folders.append(d)
    return folders 

def get_date_num(model, training_folder):
    s = training_folder.removeprefix(model+'_')
    return s.split('__training')

In [11]:
pprint(get_training_folders(cfg))

['/home/alex/workspace_local/rlrom/examples/cartpole/models/vanilla_ppo_2025_10_07__training0',
 '/home/alex/workspace_local/rlrom/examples/cartpole/models/vanilla_ppo_2025_10_07__training1']


In [6]:
print(cfg)

{'env_name': 'CartPole-v1', 'model_path': './models', 'model_name': 'vanilla_ppo', 'cfg_test': {'init_seeds': 0, 'num_ep': 10, 'num_steps': 500, 'render_mode': 'human'}, 'cfg_train': {'n_envs': 4, 'total_timesteps': 100000, 'eval_freq': 15000, 'algo': {'ppo': {'device': 'cpu', 'verbose': 0, 'tensorboard_log': './tb_logs'}}}, 'this_cfg_pathdir': '/home/alex/workspace_local/rlrom/examples/cartpole'}
